# Вебинар 6. Двухуровневые модели рекомендаций


In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [60]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [61]:
n_items_before = data_train_lvl_1['item_id'].nunique()

#параметры функции могут отличаться
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features)


n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [62]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [63]:
recommender.get_als_recommendations(2375, N=5)

[1044078, 899624, 871756, 1106523, 844179]

In [64]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [65]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 1042907, 934399, 15778319]

In [66]:
recommender.get_similar_users_recommendation(2375, N=5)

[5574377, 1101502, 963686, 10457044, 1124971]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [67]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [68]:
cold_users = np.setdiff1d(result_lvl_1['user_id'], data_train_lvl_1['user_id']).tolist()
result_lvl_1 = result_lvl_1[~result_lvl_1['user_id'].isin(cold_users)]

In [69]:
result_lvl_1['als_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=50))
result_lvl_1['own_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1['sim_items_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
##result_lvl_1['sim_user_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

result_lvl_1.head(2)

,user_id,actual,als_rec,own_rec,sim_items_rec
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[885290, 1037332, 856942, 5577022, 1082212, 10...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[824758, 1007512, 9297615, 5577022, 9803545, 9..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[916122, 5569230, 1029743, 866211, 933835, 112...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[8090537, 5569845, 1044078, 985999, 880888, 81..."


In [70]:
recall_ = pd.DataFrame()
recall_['user_id'] = result_lvl_1['user_id']
recall_['als_rec'] = result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual'], 50), axis=1)
recall_['own_rec'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual'], 50), axis=1)
recall_['sim_items_rec'] = result_lvl_1.apply(lambda x: recall_at_k(x['sim_items_rec'], x['actual'], 50), axis=1)

recall_['als_rec'].mean(), recall_['own_rec'].mean(), recall_['sim_items_rec'].mean()

(0.049991578916390805, 0.06525657038145165, 0.034169757445733376)

In [71]:
result_lvl_1['own_rec_20'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=20))
result_lvl_1['own_rec_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1['own_rec_100'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))
result_lvl_1['own_rec_200'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))
result_lvl_1['own_rec_500'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))

In [72]:
recall_['own_rec_20'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_20'], x['actual'], 20), axis=1)
recall_['own_rec_50'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_50'], x['actual'], 50), axis=1)
recall_['own_rec_100'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_100'], x['actual'], 100), axis=1)
recall_['own_rec_200'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_100'], x['actual'], 100), axis=1)
recall_['own_rec_500'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_500'], x['actual'], 500), axis=1)

In [73]:
recall_['own_rec_20'].mean(), recall_['own_rec_50'].mean(), recall_['own_rec_100'].mean(), recall_['own_rec_200'].mean(), recall_['own_rec_500'].mean()

(0.039284276793729055,
 0.06525657038145165,
 0.09604492955885016,
 0.09604492955885016,
 0.18205324555508703)

**Чем больше К - тем выше recall, ну это логично, если к будет максимальным - recall будет 1. Однако, это не улучшит качество модели. Разумнее брать маленькое число к, например 20 или 50.**

### Задание 2.

Обучите модель 2-ого уровня, при этом:
   - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
   - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker
   - Вырос ли precision@5 при использовании двухуровневой модели?

In [74]:
# your_code
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))

In [75]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [76]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [77]:
targets_lvl_2.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0


In [78]:
data_train_lvl_2 = data_train_lvl_2.merge(item_features, on='item_id', how='left')
data_train_lvl_2 = data_train_lvl_2.merge(user_features, on='user_id', how='left')

data_train_lvl_2.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,...,SOFT DRINKS,SOFT DRINK BOTTLE NON-CARB (EX,20 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,...,EGGS,EGGS - LARGE,1 DZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
#Средняя сумма покупки
def mid_sum_(df=data_train_lvl_2):
    mid_sum = df[['user_id', 'basket_id', 'sales_value']].copy()
    mid_sum = mid_sum.groupby(by=['user_id', 'basket_id']).sum()
    mid_sum = mid_sum.groupby(by=['user_id']).mean()
    mid_sum.rename(columns={"sales_value": "average_bill"}, inplace=True)

    return mid_sum

In [80]:
# Количество покупок в категории
def count_dep_(df=data_train_lvl_2):
    count_dep = df[['user_id', 'department']].copy()
    count_dep = pd.get_dummies(count_dep)
    count_dep = count_dep.groupby(by='user_id').sum()

    return count_dep

In [131]:
mid_sum = mid_sum_(df=data_train_lvl_2)
count_dep = count_dep_(df=data_train_lvl_2)

result = targets_lvl_2.merge(item_features, on=['item_id'], how='left')

result = result.merge(mid_sum, on=['user_id'], how='left')
result = result.merge(count_dep, on=['user_id'], how='left')

result.fillna(value=0, inplace=True)
result.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,average_bill,...,department_PHOTO,department_PRODUCE,department_RESTAURANT,department_RX,department_SALAD BAR,department_SEAFOOD,department_SEAFOOD-PCKGD,department_SPIRITS,department_TRAVEL & LEISUR,department_VIDEO
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,14.355581,...,0.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,14.355581,...,0.0,10.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [84]:
X_train = result.drop('target', axis=1)
y_train = result[['target']]

In [85]:
#Валидация 
users_val = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_val.columns = ['user_id']

val_users = data_val_lvl_1['user_id'].unique()
users_val = users_lvl_2[users_lvl_2['user_id'].isin(val_users)]

users_val['candidates'] = users_val['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))

In [86]:
df=pd.DataFrame({'user_id':users_val.user_id.values.repeat(len(users_val.candidates[0])),
                 'item_id':np.concatenate(users_val.candidates.values)})

In [91]:
targets_val = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_val['target'] = 1  # тут только покупки 

targets_val = df.merge(targets_val, on=['user_id', 'item_id'], how='left')

targets_val['target'].fillna(0, inplace= True)
targets_val.drop_duplicates(subset =['user_id', 'item_id'], keep = 'first', inplace = True, ignore_index=True)

In [132]:
result_val = targets_val.merge(item_features, on=['item_id'], how='left')

result_val = result.merge(mid_sum, on=['user_id'], how='left')
result_val = result.merge(count_dep, on=['user_id'], how='left')

result_val.fillna(value=0, inplace=True)
result_val.drop_duplicates(subset =['user_id', 'item_id'], keep = 'first', inplace = True, ignore_index=True)

In [133]:
result_val = result_val.reindex(columns = result.columns, fill_value=0)

In [135]:
result_val.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,average_bill,...,department_PHOTO,department_PRODUCE,department_RESTAURANT,department_RX,department_SALAD BAR,department_SEAFOOD,department_SEAFOOD-PCKGD,department_SPIRITS,department_TRAVEL & LEISUR,department_VIDEO
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,14.355581,...,0,0,0,0,0,0,0,0,0,0
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,14.355581,...,0,0,0,0,0,0,0,0,0,0


In [139]:
X_val = result_val.drop('target', axis=1)
y_val = result_val[['target']]

In [141]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [147]:
X_val[cat_feats] = X_val[cat_feats].astype('category')

In [168]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=7)
lgb.fit(X_train, y_train)

val_preds = lgb.predict_proba(X_val)[:, 1]

C:\Users\Mell\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Wall time: 1.18 s


In [171]:
X_val['predict_proba'] = val_preds

X_val.sort_values(['user_id', 'predict_proba'], ascending=False, inplace=True)

result_ = X_val.groupby('user_id').head(5)

recs = result.groupby('user_id')['item_id']
recomendations = []
for user, preds in recs:
    recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

recomendations = pd.DataFrame(recomendations)

result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(recomendations)
    

In [172]:
result_lvl_2.apply(lambda row: precision_at_k(row['recomendations'], row['actual']), axis=1).mean()

0.12490861618798955